# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Process and Transform Images

In [2]:
import pandas as pd
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class BrainTumorDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 1]

        # Encode labels
        label_dict = {'notumor': 0, 'glioma': 1, 'meningioma': 2, 'pituitary': 3}
        encoded_label = label_dict[label]

        # Load image
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, encoded_label

# Define data transformation
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the input size expected by the CNN model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize pixel values
])

In [3]:
# get your customized dataset
train_set = BrainTumorDataset(csv_file='/content/drive/MyDrive/BrainTumorClassification/train.csv', transform=data_transform)
test_set = BrainTumorDataset(csv_file='/content/drive/MyDrive/BrainTumorClassification/test.csv', transform=data_transform)

In [4]:
# Print set sizes
print(len(train_set))
print(len(test_set))

2870
394


In [5]:
from torch.utils.data import Dataset, DataLoader

batch_size=16

# Create DataLoader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [6]:
# Print loader
print(len(train_loader))
print(len(test_loader))

180
25


# Train ResNet-18 Model

In [7]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim

# Define the model
model = torchvision.models.resnet18(pretrained=True)

# Modify the last fully connected layer to fit our 4 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)
model

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
# Check if the model is on GPU
if next(model.parameters()).is_cuda:
    print("Model is currently on GPU")
else:
    print("Model is currently on CPU")

Model is currently on GPU


In [10]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
# Adam Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training phase
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Compute statistics
        running_train_loss += loss.item()
        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

    # Testing phase
    model.eval()  # Set the model to evaluation mode
    running_test_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = loss_fn(outputs, labels)

            # Compute statistics
            running_test_loss += loss.item()
            _, predicted_test = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted_test == labels).sum().item()

    # Store results in global lists
    train_losses.append(running_train_loss / len(train_loader))
    train_accuracies.append(100 * correct_train / total_train)
    test_losses.append(running_test_loss / len(test_loader))
    test_accuracies.append(100 * correct_test / total_test)

    # Print statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], "
          f"Train Loss: {train_losses[-1]:.4f}, "
          f"Train Accuracy: {train_accuracies[-1]:.2f}%, "
          f"Test Loss: {test_losses[-1]:.4f}, "
          f"Test Accuracy: {test_accuracies[-1]:.2f}%")

Epoch [1/10], Train Loss: 0.5298, Train Accuracy: 81.15%, Test Loss: 1.7041, Test Accuracy: 67.77%
Epoch [2/10], Train Loss: 0.2726, Train Accuracy: 90.21%, Test Loss: 1.4091, Test Accuracy: 73.60%
Epoch [3/10], Train Loss: 0.1888, Train Accuracy: 93.10%, Test Loss: 1.7190, Test Accuracy: 72.34%
Epoch [4/10], Train Loss: 0.1628, Train Accuracy: 94.22%, Test Loss: 1.7934, Test Accuracy: 69.29%
Epoch [5/10], Train Loss: 0.1213, Train Accuracy: 95.47%, Test Loss: 1.6977, Test Accuracy: 70.81%
Epoch [6/10], Train Loss: 0.0977, Train Accuracy: 96.76%, Test Loss: 1.7979, Test Accuracy: 74.11%
Epoch [7/10], Train Loss: 0.1063, Train Accuracy: 96.83%, Test Loss: 1.9111, Test Accuracy: 59.14%
Epoch [8/10], Train Loss: 0.0611, Train Accuracy: 97.74%, Test Loss: 1.4859, Test Accuracy: 75.63%
Epoch [9/10], Train Loss: 0.0498, Train Accuracy: 98.15%, Test Loss: 3.2180, Test Accuracy: 71.83%
Epoch [10/10], Train Loss: 0.0722, Train Accuracy: 97.39%, Test Loss: 1.2825, Test Accuracy: 76.65%
